# English Premiership League 2020/21 Total Goals Scored Visualization

## Import Required Libraries

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt # plotting library
import seaborn as sns # high-level visualization library

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

## Get file path

In [2]:
file = ""
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        file = os.path.join(dirname, filename)

## Read Data

In [3]:
data = pd.read_excel(file)

In [4]:
data.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,2020-09-12,12:30:00,Fulham,Arsenal,0,3,A,0,1,...,1.84,0.75,2.01,1.89,2.02,1.91,2.13,1.92,2.02,1.87
1,E0,2020-09-12,15:00:00,Crystal Palace,Southampton,1,0,H,1,0,...,1.70,0.25,1.78,2.13,1.79,2.17,1.85,2.18,1.79,2.12
2,E0,2020-09-12,17:30:00,Liverpool,Leeds,4,3,H,3,2,...,2.62,-1.50,1.85,2.05,1.85,2.08,1.90,2.16,1.84,2.04
3,E0,2020-09-12,20:00:00,West Ham,Newcastle,0,2,A,0,0,...,1.92,-0.50,2.03,1.87,2.04,1.88,2.09,1.91,2.02,1.86
4,E0,2020-09-13,14:00:00,West Brom,Leicester,0,3,A,0,0,...,1.73,0.25,1.92,1.98,1.93,1.99,1.95,2.01,1.91,1.97


In [5]:
# choose required columns
select_columns = data.columns[3:7]
select_columns

Index(['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG'], dtype='object')

In [6]:
new_data = data[select_columns]
new_data.head()

,HomeTeam,AwayTeam,FTHG,FTAG
0,Fulham,Arsenal,0,3
1,Crystal Palace,Southampton,1,0
2,Liverpool,Leeds,4,3
3,West Ham,Newcastle,0,2
4,West Brom,Leicester,0,3


## Get combination of team play home and away

In [7]:
clubs = pd.concat([new_data['HomeTeam'], new_data['AwayTeam']], axis=0)

## Create Premier League Table

In [8]:
table = pd.DataFrame(zip(clubs.value_counts().index, clubs.value_counts().values), columns=['club', 'played'])

# Add other columns as placeholders
other_columns = ["Won", "Drawn", "Lost", "GF", "GA", "GD", "Points"]
for col in other_columns:
    table[col] = 0
    
table.head()

,club,played,Won,Drawn,Lost,GF,GA,GD,Points
0,Fulham,38,0,0,0,0,0,0,0
1,Crystal Palace,38,0,0,0,0,0,0,0
2,Burnley,38,0,0,0,0,0,0,0
3,Wolves,38,0,0,0,0,0,0,0
4,Aston Villa,38,0,0,0,0,0,0,0
